In [439]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [440]:
stories_json = []
with open('./output_stories_0123.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

839
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [441]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_title_joined"] = stories_df["title"].str.join(" ") + " " + stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_title_joined
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,「 映 画 ド ラ え も ん の び 太 の 新 魔 界 大 冒 険 ～ ７ 人 の ...
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,「 ば く は つ コ シ ョ ウ 」 ある朝、ねぼうしたのび太に泣きつかれたドラえもんは...
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,「 何 が 何 で も お 花 見 を 」 みんなから家族でお花見に行った話を聞き、うらや...
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,「 や り ク リ し て ハ ワ イ 旅 行 」 スネ夫からハワイ旅行をじまんされたのび...
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,「 し ず か ち ゃ ん の は ご ろ も 」 学校で「天女と羽衣（はごろも）」の劇（...


In [442]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞", "形容詞", "動詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

wt = MeCab.Tagger('-Owakati')
def mecab_tokenizer_wakachi(text):
    return wt.parse(text).split(" ")

In [443]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_title_joined'].str.len() > 0] 
sentences = stories_df["abstract_title_joined"].apply(mecab_tokenizer_wakachi)
print(sentences)

0      [「, 映, 画, ド, ラ, え, も, ん, の, び, 太, の, 新, 魔, 界, ...
1      [「, ば, く, は, つ, コ, シ, ョ, ウ, 」, 　, ある, 朝, 、, ね,...
2      [「, 何, が, 何, で, も, お, 花, 見, を, 」, 　, みんな, から, ...
3      [「, や, り, ク, リ, し, て, ハ, ワ, イ, 旅, 行, 」, 　, スネ,...
4      [「, し, ず, か, ち, ゃ, ん, の, は, ご, ろ, も, 」, 　, 学校,...
                             ...                        
834    [「, 夜, 空, に, 輝, く, ピ, ザ, ・, ギ, ョ, ー, ザ, 」, 星, ...
835    [「, あ, り, が, た, み, わ, か, り, 機, 」, おなか, が, いっぱい...
836    [「, の, び, 泥, 棒, を, タ, イ, ホ, せ, よ, ！, 」, 小さな, 子...
837    [「, 鬼, の, パ, ン, ツ, は, い, い, パ, ン, ツ, ？, 」, ５, ...
838    [「, 雪, だ, る, ま, が, 町, に, や, っ, て, き, た, 」, 冬, ...
Name: abstract_title_joined, Length: 839, dtype: object


In [444]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [445]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[-30.025955   15.140804 ]
 [ -0.5854167 -16.529814 ]
 [  8.780884   -4.6549277]
 ...
 [-33.514553    4.7760115]
 [  5.49508    -1.0748798]
 [ -1.7304354 -11.279624 ]]


In [446]:
vector_df = pd.DataFrame.from_dict({
    'vector': [v for v in vector_list],
    'vector_x': [v[0] for v in vector_enbedded],
    'vector_y': [v[1] for v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year
stories_df_merged.reset_index().to_csv("stories_df_merged.csv")


In [447]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [448]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()

In [449]:
sampled_df = stories_df_merged.sample(1)
print(sampled_df.head())
sampled_df = sampled_df.rename(columns={"vector": "sampled_vec"})[["sampled_vec"]]


    index  stories_index        title broadcasting_date  \
404  0527              0  「録験機でたのしもう」        2018年6月29日   

                                        abstracts_list  \
404  [帰宅したのび太が部屋に入ると、頭にヘッドホンをしたドラえもんが、手には何も持っていないのに...   

                                 abstract_title_joined  \
404  「 録 験 機 で た の し も う 」 帰宅したのび太が部屋に入ると、頭にヘッドホンをし...   

                                                vector  vector_x   vector_y  \
404  [-0.101601936, 0.10842953, -0.04275749, 0.0781... -2.592682  13.755471   

    broadcasting_dt  broadcasting_year  
404      2018-06-29               2018  


In [450]:
stories_df_merged = stories_df_merged.assign(key=1).merge(sampled_df.assign(key=1), on='key').drop('key', axis=1)


In [451]:
from scipy.spatial.distance import cosine

stories_df_merged["cosine_similarity"] =  stories_df_merged.apply(lambda v: 1 - cosine(v["vector"], v["sampled_vec"]), axis=1)
stories_df_merged_sort = stories_df_merged[["title", "abstract_title_joined", "cosine_similarity"]].sort_values("cosine_similarity", ascending=False)


In [452]:
stories_df_merged_sort

,title,abstract_title_joined,cosine_similarity
404,「録験機でたのしもう」,「 録 験 機 で た の し も う 」 帰宅したのび太が部屋に入ると、頭にヘッドホンをし...,1.000000
104,「神さまごっこ」,「 神 さ ま ご っ こ 」 \r\n 「もう３日もどら焼きを食べていない」となげくドラえ...,0.899910
774,「うつしっぱなしミラー」,「 う つ し っ ぱ な し ミ ラ ー 」 お昼休み、校庭でみんなと遊んでいたのび太は、...,0.897968
230,「チリつもらせ機」,「 チ リ つ も ら せ 機 」 \r\n 今日食べるどら焼きがないことに気づいたドラえも...,0.864847
480,「ホ、ホ、ホタル来い」,「 ホ 、 ホ 、 ホ タ ル 来 い 」 暑い夏の日、プールに行こうと思いつくのび太だった...,0.858844
...,...,...,...
752,「泣くなジャイ子よ」,「 泣 く な ジ ャ イ 子 よ 」 のび太が空き地で昼寝（ね）をしていると、何やらジャイ...,0.329026
372,「大ピンチ！スネ夫の答案」,「 大 ピ ン チ ！ ス ネ 夫 の 答 案 」 久しぶりにテストで百点を取ったスネ夫は...,0.320580
66,「映画ドラえもん のび太のひみつ道具博物館（ミュージアム）」,「 映 画 ド ラ え も ん の び 太 の ひ み つ 道 具 博 物 館 （ ミ ...,0.320279
750,「つかめドリーム！宝星」,「 つ か め ド リ ー ム ！ 宝 星 」 学校の裏（うら）山から、財宝（ざいほう）が見...,0.239548
